### Enter full names of group members:

##### Name A: Nick Askari
##### Name B: Simen Peder Stang

In [106]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [107]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [108]:

def dgim_algorithm(stream_path, N):
    # Buckets list initialization
    buckets = []
    # Current time in the stream
    timestamp = 0

    # Open the file containing the stream
    with open(stream_path, 'r') as f:
        while True:

            bit = f.read(1)
            if not bit:
                break

            timestamp = (timestamp + 1) % N

            # Remove buckets outside the N-bit window

            buckets = [bucket for bucket in buckets if bucket[1] != timestamp]

            if bit == '1':
                buckets.append((1, timestamp))

                # Merging buckets
                i = 0
                while i < len(buckets) - 2:
                    # Find three consecutive buckets of the same size
                    if buckets[i][0] == buckets[i + 1][0] and buckets[i][0] == buckets[i + 2][0]:
                        # Merge the first two
                        new_size = buckets[i][0] * 2
                        new_timestamp = buckets[i + 1][1]
                        buckets[i + 1] = (new_size, new_timestamp)
                        del buckets[i]
    
                        i = 0
                    else:
                        i += 1


    # Prepare output list
    bucket_list = [[] for _ in range(math.ceil(math.log2(N)))]
    end_time_stamp = None

    # Fill output list and determine end timestamp
    for i in range(len(buckets)):
        size, ts = buckets[i][0], buckets[i][1]
   
        index = int(math.log(size) / math.log(2))
        if index < len(bucket_list):
            bucket_list[index].append(ts)
            bucket_list[index].sort()
            if size == 1:
                end_time_stamp = ts

    return bucket_list, end_time_stamp

In [109]:
bucket = dgim_algorithm(stream_path, N)

In [110]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188], []]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [111]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [112]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    one_count = 0
    
    last_bucket = 0
    for exponent, timestamps in enumerate(bucket_list):
        bucket_size = 2 ** exponent
        for timestamp in timestamps:
            # Calculate the effective range of timestamps to consider
            if (end_time_stamp - timestamp) % N < k:
                one_count += bucket_size
                last_bucket = exponent
    
    # Adding only half the size of the last bucket (as in the lecture)
    if last_bucket:
        one_count -= 2**(last_bucket)/2

    return math.ceil(one_count)

In [113]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [114]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [297]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [298]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [299]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [300]:
def generate_hash(h, N):
    hash_list = []
    
    # To-do! generate a list of hash functions
    prime_dict = {}

    for _ in range(h):
        random_prime = prime(random.randint(1, 100 * h))
        while random_prime in prime_dict:
            random_prime = prime(random.randint(1, 100 * h))
        prime_dict[random_prime] = True

        hash_function = lambda s, p=random_prime, N=N: sum((ord(c) * p**i for i, c in enumerate(s))) % N
        hash_list.append(hash_function)
    
    return hash_list

In [301]:
hashes = generate_hash(h, bloom_size)

In [302]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            
            # To-do! update the hash index of the bloom filter with 1s
            for hash_function in hashes:
                B[hash_function(name)] = 1
            
    return B

In [303]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [304]:
bloom_array

array([1., 0., 1., ..., 0., 0., 0.])

#### 2.2. Verify usernames

In [305]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash_func in hashes:
        index = hash_func(new_user)
        if B[index] != 1:
            code = 0
            break

        code = 1
        
    return code

In [306]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [307]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [308]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [310]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            # To-do! similar to the single verify, but returns a percentage of usernames taken...
            # ...(In other words seen already by the bloom filter during its creation)
            
            taken_name += single_verify_username(bloom_array, hashes, name)
            total_name += 1

            
    return round(taken_name/total_name*100,2)   

In [311]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.93%
----------------------------------------------------------


original. We have correct. The hash functions leads a little bit of randomness
----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.71%
----------------------------------------------------------

### 3. Flajolet-Martin

In [320]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    hash_function = lambda x: 6*x +  1 % 5

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    for i in input_stream:
        binary_string = format(i, 'b')

        r = 0
        for bit in reversed(binary_string):
            if bit == '0':
                r += 1
            else:
                break
        
        if r > R:
            R = r

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [321]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [342]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [343]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [346]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    for query in queries:
        potential_advertisers = []

        for company, i in local_companies.items():
            keywords, budget = i[:-1], i[-1]
            
            if query in keywords:
                if budget > 0:
                    potential_advertisers.append(company)
            
        if len(potential_advertisers) > 0:
            chosen_bidder = random.choice(potential_advertisers)
            revenue += 1

            local_companies[chosen_bidder][-1] -= 1
    
    return revenue

In [453]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 7
Trial 2 - Revenue generated: 8
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 8
Trial 5 - Revenue generated: 8
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 8
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 9
------------------------------------------------
Average revenue generated for all trials:  8.4


#### 4.2. Balance Algorithm

In [410]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # We have to choose advertisers with the highest budgets
    for query in queries:
        potential_advertisers = []

        for company, i in local_companies.items():
            keywords, budget = i[:-1], i[-1]
            
            if query in keywords:
                if budget > 0:
                    potential_advertisers.append((company, budget))
        
            
        if len(potential_advertisers) > 0:
            potential_advertisers = sorted(potential_advertisers, key=lambda x: x[1], reverse=True)
            highest_budget = potential_advertisers[0][1]

            # potential_advertisers are all the companies with the highest budgets now.
            potential_advertisers = [comp[0] for comp in potential_advertisers if comp[1] == highest_budget]

            chosen_bidder = random.choice(potential_advertisers)
            revenue += 1

            local_companies[chosen_bidder][-1] -= 1
    
    return revenue

In [482]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 9
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 10
Trial 10 - Revenue generated: 9
-------------------------------------------
Average revenue generated for all trials:  9.3


### 5. Recommender System

In [ ]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [ ]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    
    return prediction   

In [ ]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [ ]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

One has to also consider that each of the buckets used store O(logn) bits for the timestamp. Hence if there are logn buckets, and each of these have a timestamp that is 
stored with O(logn) bits, this results in a space complexity of O(log^2(N)).

#### Case 2

If the bloom filter outputs that "Kazeem" is taken then their is a certain possibility that this is a false positive. This is because with bloom filters, although with create memory usage, it could give out false positives. I would additionally tell the admin that this is highly correlated to the use of hash functions. It is possible that somehow all the hash functions consider a username to be taken, due to imperfected hashes. 

In this case the bloom filter outputs that a username has NOT been taken. A bloom filter guarantees no have false negatives, hence it is impossible that the username "KazeemTDT4305" has been taken. For the bloom filter to return that a username has NOT been taken, only one of the indexes given by one hash function in the bloom filter must be 0. This will not happen if the username was taken. 

The reason that Google finds that particular email to be taken could be because of false positives when using bloom fitlers as discussed.

#### Case 3

The question is how to increase precision while using the Flaholet-Martin algorithm. This can be done by including several hash functions, and then distributing them into multiple groups. So now you have groups of hash functions. A value from the input stream will now get a value from all these groups. Within each group, the median value out of the hash functions will be chosen. Lastly one takes the average of all the median values obtained from different groups.

#### Case 4

### Maximum possible revenue
For example as such,

- Company D gets all of the queries regarding 'big data'. We have earned 3 dollars then. Company D has no budget left.
- Company A gets all of the queries regarding 'bloom filters'. Now we have earned 3 + 3 = 6 dollars. Company A has no budget left.
- Company B gets all of the queries regarding 'flajolet martin'. Now we have earned 3 + 3 + 3 = 9 dollars. Company B has no budget left.
- Company C gets all of the queries regarding 'dgim algorithm'. Now we have earned 3 + 3 + 3 + 3 = 12 dollars. Company C has no budget left.

Such a matching yields a **revenue of 12 dollars.**

### Minimum possible revenue
- Company A gets all of the queries regarding 'big data'. We have earned 3 dollars then. Company A has no budget left.
- Company C gets all of the queries regarding 'flajolet martin'. Now we have earned 3 + 3 = 6 dollars. Company C has no budget left.

The user queries left are not wanted by either company B or company D. This outcome yields the minimum revenue, **6 dollars**.

### Competitive ratio for the Greedy algorithm
We must find the greedy's worst performance. The greedy algorithm picks all the bidders with a relevant keyword (and positive budget). Amongst these, the ultimate bidder is picked randomly. It is possible that it randomly manages to choose the solution with minimum revenue. Hence the greedy algorithms worst performance yields a revenue of 6.

Competitive ratio becomes,

\begin{align}
\text{Competitive ratio}_{greedy} = \frac{6}{12} = \underline{\frac{1}{2}}
\end{align}

### Competitive ratio for the Balance algorithm
The worst possible performance with the balance algorithm, yields the following.

- Company A gets 2 x 'big data' and 1 x 'bloom filter'. This yields a revenue of 3. Company A's budget is now 0.
- Company B gets 1 x 'flajolet martin'. This yields a revenue of 1. Company B has 2 left in budget.
- Company C gets 2 x 'flajolet martin' and 1 x 'dgim algorithm'. This yields a revenue of 3. Company C's budget is now 0.
- Company D gets 1 x 'big data'. This yields a revenue of 1. Company D has 2 left in budget.

Adding up all the revenues we get,

\begin{align}
    R = 3 + 1 + 3 + 1 = \underline{8}
\end{align}

Competitive ratio becomes,

\begin{align}
\text{Competitive ratio}_{balance} = \frac{8}{12} = \underline{\frac{2}{3}}
\end{align}

#### Case 5

In [ ]:
# Enter answer here